In [1]:
import torch
import torch.nn as nn
import pandas as pd

In [2]:
#! 暂时只考虑坏地的情况
A_year_len = 8
A_land_len = 26
A_crop_len = 15

## 耕地性质, 作物性质

In [3]:
land_df = pd.read_excel("耕地性质.xlsx")
crop_df = pd.read_excel("农作物性质.xlsx")

land_name2idx = {}
land_idx2name = {}
land_idx2type = {}
land_idx2area = {}
for i in range(len(land_df)):
    land_name = land_df.iloc[i]['地块名称'].strip()
    land_type = land_df.iloc[i]['地块类型'].strip()
    land_area = land_df.iloc[i]['地块面积/亩']
    land_name2idx[land_name] = i
    land_idx2name[i] = land_name
    land_idx2type[i] = land_type
    land_idx2area[i] = land_area
    
crop_name2idx = {}
crop_idx2name = {}
crop_idx2type = {}
crop_idx2land = {}
for i in range(len(crop_df)):
    crop_name = crop_df.iloc[i]['作物名称'].strip()
    crop_type = crop_df.iloc[i]['作物类型'].strip()
    crop_land = crop_df.iloc[i]['种植耕地'].strip()
    crop_name2idx[crop_name] = i
    crop_idx2name[i] = crop_name
    crop_idx2type[i] = crop_type
    crop_idx2land[i] = crop_land

del land_df, crop_df, i
del land_name, land_type, land_area, crop_name, crop_type, crop_land

## 销售价格 `price_ts`

In [4]:
price_2023 = pd.read_excel("2023年销售数据.xlsx")

# 计算每种作物的每亩均值利润
for i in range(len(price_2023)):
    price_range = price_2023.loc[i, "销售单价/(元/斤)"]
    price_low = price_range.split("-")[0]
    price_high = price_range.split("-")[1]
    price_avg = (float(price_low) + float(price_high)) / 2
    price_2023.loc[i, "均值单价"] = round(price_avg, 4)
    price_2023.loc[i, "均值利润"] = price_2023.loc[i, "亩产量/斤"] * price_avg - price_2023.loc[i, "种植成本/(元/亩)"]
    price_2023.loc[i, "均值利润"] = price_2023.loc[i, "均值利润"].round(4)

del price_range, price_low, price_high, price_avg, i

In [5]:
import sqlite3

# 建立数据库 <作物索引, 地块类型, 季次, 每亩利润>
conn = sqlite3.connect("price.db")
price_db = conn.cursor()
price_db.execute("DROP TABLE IF EXISTS price_tb")
price_db.execute("CREATE TABLE IF NOT EXISTS price_tb (crop_idx INTEGER, land_type TEXT, season TEXT, price REAL)")
for i in range(len(price_2023)):
    crop_name = price_2023.loc[i, "作物名称"].strip()
    crop_idx = crop_name2idx[crop_name]
    land_type = price_2023.loc[i, "地块类型"].strip()
    season = price_2023.loc[i, "种植季次"].strip()
    price = price_2023.loc[i, "均值利润"]
    price_db.execute("INSERT INTO price_tb VALUES (?, ?, ?, ?)", (crop_idx, land_type, season, price))
    if land_type == "普通大棚":
        price_db.execute("INSERT INTO price_tb VALUES (?, ?, ?, ?)", (crop_idx, "智慧大棚", season, price))

conn.commit()
del crop_name, crop_idx, land_type, season, price, i, conn

In [6]:
# 构建销售利润价格矩阵 <地块索引, 作物索引>-<价格利润>
price_ts = torch.zeros(A_land_len, A_crop_len)
for i in range(A_land_len):
    land_type = land_idx2type[i]
    for j in range(A_crop_len):
        price_db.execute("SELECT price FROM price_tb WHERE crop_idx = ? AND land_type = ?", (j, land_type)) #! 暂时只考虑单季
        crop_price = price_db.fetchone()
        if crop_price is not None:
            price_ts[i, j] = crop_price[0]

## 耕地面积限制 `capat_ts`

In [7]:
capat_ts = torch.tensor([land_idx2area[i] for i in range(A_land_len)])

## 作物销量 `demand_ts`

In [8]:
# 假设需求量是23年产量的SALE2VOL倍
SALE2VOL = 1.3

In [9]:
plant_2023 = pd.read_excel("2023年种植数据.xlsx")
demand_ts = torch.zeros(A_crop_len)
for i in range(len(plant_2023)):
    crop_idx = crop_name2idx[plant_2023.loc[i, "作物名称"].strip()]
    crop_sale = plant_2023.loc[i, "种植面积/亩"]
    if crop_idx < A_crop_len:
        demand_ts[crop_idx] += crop_sale * SALE2VOL

del plant_2023, crop_idx, crop_sale

## 模型构建

In [10]:
class ProfitModel(nn.Module):
    def __init__(self, year_len: int, land_len: int, crop_len: int, price_ts: torch.Tensor, capat_ts: torch.Tensor, demand_ts: torch.Tensor, device: str, focus_mask: float):
        super(ProfitModel, self).__init__()
        self.device = device
        self.land_len = land_len
        self.crop_len = crop_len
        self.plant_ts = nn.Parameter(torch.zeros(land_len, crop_len).to(torch.float32))  # 每块地的种植面积需要训练

        assert price_ts.shape == (land_len, crop_len)
        self.price_ts = price_ts.transpose(0, 1).to(device)  # 销售价格已确定, 无需训练

        assert capat_ts.shape == (land_len,)
        self.capat_ts = capat_ts.to(device)  # 每块地的种植面积上限已确定, 无需训练
        self.plant_mask = (self.capat_ts * focus_mask).unsqueeze(1).to(torch.float32)

        assert demand_ts.shape == (crop_len,)
        self.demand_ts = demand_ts.to(device)  # 每种作物的需求量已确定, 无需训练

    def forward(self):
        # 如果plant_ts的种植过于分散, 则收益减小
        torch.where(self.plant_ts < self.plant_mask, self.plant_ts / 10.0, self.plant_ts)
        # 矩阵相乘, 并取对角线元素之和
        profit_ts = torch.matmul(self.plant_ts, self.price_ts)
        profit_sum = profit_ts.diag().sum()
        # 如果某块耕地的种植面积超过了上限, 则惩罚
        profit_sum -= torch.relu(self.plant_ts.sum(dim=1) - self.capat_ts).sum() * 1e9
        # 如果某种作物的产量超过了需求量, 则惩罚
        profit_sum -= torch.relu(self.plant_ts.transpose(0, 1).sum(dim=1) - self.demand_ts).sum() * 1e9
        return -profit_sum

In [11]:
focus_mask = 0.2
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")
model = ProfitModel(A_year_len, A_land_len, A_crop_len, price_ts, capat_ts, demand_ts, device, focus_mask).to(device)

Using device: cuda


In [12]:
keep_vars = ["model", "crop_idx2name", "A_crop_len", "land_idx2name", "A_land_len"]
for var in list(globals().keys()):
    if var not in keep_vars and not var.startswith("_"):
        del globals()[var]
del var

In [19]:
# 开始进行训练
import torch
optimizer = torch.optim.Adam(model.parameters(), lr=0.00001, weight_decay=0.01)
for epoch in range(50000):
    optimizer.zero_grad()
    loss = model()
    loss.backward()
    optimizer.step()
    if epoch % 500 == 0:
        print(f"Epoch {epoch}, Loss {-loss.item()}")

Epoch 0, Loss 2521880.0
Epoch 500, Loss 2523040.25
Epoch 1000, Loss 2523040.25
Epoch 1500, Loss 2523040.25
Epoch 2000, Loss 2523040.25
Epoch 2500, Loss 2523040.25
Epoch 3000, Loss 2523040.25
Epoch 3500, Loss 2523040.25
Epoch 4000, Loss 2523040.25
Epoch 4500, Loss 2523040.25
Epoch 5000, Loss 2523040.25
Epoch 5500, Loss 2523040.25
Epoch 6000, Loss 2523040.25
Epoch 6500, Loss 2523040.25
Epoch 7000, Loss 2523042.75
Epoch 7500, Loss 2523050.75
Epoch 8000, Loss 2523060.0
Epoch 8500, Loss 2523074.25
Epoch 9000, Loss 2523094.0
Epoch 9500, Loss 2523080.75
Epoch 10000, Loss 2523073.5
Epoch 10500, Loss 2523048.25
Epoch 11000, Loss 2522833.0
Epoch 11500, Loss 2522833.0
Epoch 12000, Loss 2522833.0
Epoch 12500, Loss 2522833.0
Epoch 13000, Loss 2522833.0
Epoch 13500, Loss 2522833.0
Epoch 14000, Loss 2522833.0
Epoch 14500, Loss 2522833.0
Epoch 15000, Loss 2522833.0
Epoch 15500, Loss 2522833.0
Epoch 16000, Loss 2522833.0
Epoch 16500, Loss 2522833.25
Epoch 17000, Loss 2522835.0
Epoch 17500, Loss 2522840

In [20]:
import pandas as pd
plant_ts = model.plant_ts.cpu().detach().numpy()
plant_df = pd.DataFrame(plant_ts, columns=[crop_idx2name[i] for i in range(A_crop_len)], index=[land_idx2name[i] for i in range(A_land_len)])